# 02 Data Quality Check — ASEAN Carbon Emission (2000–2024)
Notebook ini berfokus pada pengecekan kualitas data setelah proses filtering (ASEAN, 2000–2024).

Tujuannya adalah memastikan dataset layak dipakai untuk analisis tanpa menghasilkan kesimpulan yang bias akibat masalah data.
Fokus pemeriksaan:
- ukuran data dan kolom penting
- missing values
- duplikasi data
- konsistensi kombinasi country dan year
- cakupan tahun per negara

In [8]:
import pandas as pd
import numpy as np


Dataset hasil olahan dari notebook 01 dipanggil dari folder data/process. 
Langkah ini dilakukan agar proses analisis tidak mengulang download data mentah dan filter dari awal.

In [11]:
df = pd.read_csv("data/process/owid_co2_asean_2000_2024.csv")
df.shape


(250, 79)

Output shape menunjukkan ukuran dataset setelah filtering.
Dataset ini akan menjadi dasar untuk seluruh analisis pada notebook berikutnya.

## Quick Overview
Bagian ini mengecek struktur dasar dataset untuk memastikan kolom kunci tersedia dan tipe data terlihat masuk akal.

In [12]:
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Brunei,2000,BRN,326429.0,NaN,0.0,0.0,5.886,-0.092,-1.537,...,NaN,0.017,0.0,0.0,0.0,0.0,9.218,8.123,-2.344,-39.830
1,Brunei,2001,BRN,333353.0,NaN,0.0,0.0,5.758,-0.128,-2.178,...,NaN,0.017,0.0,0.0,0.0,0.0,9.554,8.428,-2.237,-38.852
2,Brunei,2002,BRN,340108.0,NaN,0.0,0.0,5.285,-0.473,-8.206,...,NaN,0.017,0.0,0.0,0.0,0.0,8.517,7.424,-1.717,-32.479
3,Brunei,2003,BRN,346650.0,NaN,0.0,0.0,6.140,0.854,16.162,...,NaN,0.018,0.0,0.0,0.0,0.0,9.600,8.381,-0.516,-8.412
4,Brunei,2004,BRN,352921.0,NaN,0.0,0.0,5.967,-0.173,-2.817,...,NaN,0.018,0.0,0.0,0.0,0.0,8.826,7.879,-0.508,-8.519


Beberapa baris awal membantu memastikan data sudah sesuai scope (ASEAN dan tahun 2000–2024).

Pemeriksaan kolom dan tipe data dilakukan untuk melihat:
- kolom apa saja yang tersedia
- tipe data setiap kolom
- indikasi awal missing values

In [13]:
df.columns


Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

Daftar kolom ini membantu menentukan metrik apa saja yang mungkin relevan untuk analisis emisi.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    250 non-null    object 
 1   year                                       250 non-null    int64  
 2   iso_code                                   250 non-null    object 
 3   population                                 250 non-null    float64
 4   gdp                                        207 non-null    float64
 5   cement_co2                                 250 non-null    float64
 6   cement_co2_per_capita                      250 non-null    float64
 7   co2                                        250 non-null    float64
 8   co2_growth_abs                             250 non-null    float64
 9   co2_growth_prct                            250 non-null    float64
 10  co2_including_luc         

Ringkasan tipe data dan jumlah non-null per kolom digunakan untuk mendeteksi kolom yang banyak missing values.

Jika banyak kolom memiliki non-null rendah, maka analisis pada kolom tersebut perlu diberi catatan atau dipilih ulang metrik yang lebih lengkap.

## Missing Values Check
Missing values dapat menyebabkan analisis tren dan agregasi menjadi tidak akurat.
Langkah ini menghitung jumlah missing values per kolom untuk melihat kolom mana yang paling bermasalah.

In [15]:
df.isnull().sum().sort_values(ascending=False)


,0
other_co2_per_capita,250
share_global_other_co2,250
share_global_cumulative_other_co2,250
other_industry_co2,250
cumulative_other_co2,250
...,...
temperature_change_from_co2,0
temperature_change_from_n2o,0
temperature_change_from_ghg,0
total_ghg_excluding_lucf,0


Kolom dengan missing values tinggi sebaiknya tidak langsung dipakai untuk insight, kecuali ada alasan kuat atau rencana imputasi.

Agar lebih mudah membandingkan tingkat missing antar kolom, persentase missing values dihitung terhadap total baris data.

In [16]:
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_percentage.sort_values(ascending=False)


,0
other_co2_per_capita,100.0
share_global_other_co2,100.0
share_global_cumulative_other_co2,100.0
other_industry_co2,100.0
cumulative_other_co2,100.0
...,...
temperature_change_from_co2,0.0
temperature_change_from_n2o,0.0
temperature_change_from_ghg,0.0
total_ghg_excluding_lucf,0.0


Persentase missing membantu menentukan threshold seperti kolom dengan missing > 50% dipertimbangkan untuk tidak dipakai pada analisis utama

## Duplicate Check
Duplikasi bisa muncul karena proses penggabungan data atau data sumber.
Langkah ini mengecek apakah ada baris yang identik sepenuhnya

In [17]:
df.duplicated().sum()


np.int64(0)

pada dataset ini tidak ada data yang bersifat duplikat.

Selain cek duplikat tiap value, hal yang lebih penting adalah memastikan kombinasi country dan year unik.
Secara logika, satu negara untuk satu tahun seharusnya hanya punya satu baris data.

Karena jika ada duplikasi pada country-year, analisis tren per negara akan terganggu karena satu tahun dihitung lebih dari sekali.

In [18]:
df.duplicated(subset=["country", "year"]).sum()


np.int64(0)

## Coverage Check per Country
Bagian ini memeriksa apakah setiap negara ASEAN memiliki jumlah tahun yang relatif seimbang dalam rentang 2000–2024.
Jika ada negara dengan jumlah tahun jauh lebih sedikit, maka analisis perbandingan perlu diberi catatan.

In [19]:
df.groupby("country")["year"].nunique().sort_values(ascending=False)


,year
country,
Brunei,25
Cambodia,25
Indonesia,25
Laos,25
Malaysia,25
Myanmar,25
Philippines,25
Singapore,25
Thailand,25


Untuk memastikan scope tahun benar-benar sesuai, minimum dan maksimum tahun dicek kembali.

Jika output tidak menunjukkan (2000, 2024), maka proses filter atau proses load processed dataset perlu diperiksa ulang.

In [20]:
df["year"].min(), df["year"].max()


(2000, 2024)

## Selecting Analysis-Ready Columns
Dataset memiliki banyak kolom, namun tidak semua kolom layak digunakan untuk analisis utama.
Beberapa kolom memiliki missing values yang sangat tinggi sehingga berisiko menghasilkan insight yang bias.
Pada bagian ini, kolom akan diseleksi berdasarkan:
- tingkat kelengkapan data
- relevansi umum terhadap analisis emisi karbon
- kemudahan interpretasi

In [21]:
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_percentage.sort_values()


,0
country,0.0
year,0.0
iso_code,0.0
population,0.0
cement_co2,0.0
...,...
cumulative_other_co2,100.0
other_co2_per_capita,100.0
other_industry_co2,100.0
share_global_cumulative_other_co2,100.0


Output menunjukkan kolom dengan persentase missing values paling rendah hingga paling tinggi.
Kolom dengan missing values yang rendah lebih stabil untuk dilakukan analisis tren dan perbandingan.

Sebagai langkah awal, dibuat threshold sederhana untuk menentukan kolom yang cukup lengkap.
Kolom dengan missing values <= 10% dianggap cukup layak untuk analisis eksploratif.

In [22]:
analysis_ready_cols = missing_percentage[missing_percentage <= 10].index.tolist()
len(analysis_ready_cols), analysis_ready_cols


(55,
 ['country',
  'year',
  'iso_code',
  'population',
  'cement_co2',
  'cement_co2_per_capita',
  'co2',
  'co2_growth_abs',
  'co2_growth_prct',
  'co2_including_luc',
  'co2_including_luc_growth_abs',
  'co2_including_luc_growth_prct',
  'co2_including_luc_per_capita',
  'co2_including_luc_per_unit_energy',
  'co2_per_capita',
  'co2_per_unit_energy',
  'cumulative_cement_co2',
  'cumulative_co2',
  'cumulative_co2_including_luc',
  'cumulative_flaring_co2',
  'cumulative_luc_co2',
  'cumulative_oil_co2',
  'energy_per_capita',
  'flaring_co2',
  'flaring_co2_per_capita',
  'ghg_excluding_lucf_per_capita',
  'ghg_per_capita',
  'land_use_change_co2',
  'land_use_change_co2_per_capita',
  'methane',
  'methane_per_capita',
  'nitrous_oxide',
  'nitrous_oxide_per_capita',
  'oil_co2',
  'oil_co2_per_capita',
  'primary_energy_consumption',
  'share_global_cement_co2',
  'share_global_co2',
  'share_global_co2_including_luc',
  'share_global_cumulative_cement_co2',
  'share_global_

Output ini menunjukkan jumlah kolom yang relatif lengkap dibandingkan total kolom dalam dataset namun tidak semua kolom ini akan dipakai, tetapi daftar ini menjadi kandidat utama.